<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_10_inventory_forecast_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 📘 Day 10 — Event Host: Inventory Forecast Tool

## 🧠 Теория

### 🟩 VectorStoreIndex (FAISS) в LlamaIndex 0.10+
```python
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader

docs = SimpleDirectoryReader("data").load_data()
faiss_store = FaissVectorStore(dim=1536)
storage_context = StorageContext.from_defaults(vector_store=faiss_store)
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)
```


## 🛠️ Практика

### Цель:
Построить систему прогнозирования по Excel-файлу `sales_data.xlsx`, с раздельным анализом по регионам.

---

### 🧾 Шаги:
1. Загрузить и парсить Excel
2. Разделить данные по `Region`
3. Построить VectorStoreIndex на основе данных
4. Создать отдельные цепочки/ветки анализа
5. Сформировать markdown-отчет

---

## 🧪 Код-заготовки



In [18]:
!pip install llama-index faiss-cpu openai
!pip install llama-index-vector-stores-faiss
!pip install faiss-cpu

In [19]:
import pandas as pd
import faiss
from datetime import datetime
from llama_index.core import Document, VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import SubQuestionQueryEngine

In [10]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [11]:
# 📥 Загрузка Excel-файла
# TODO: загрузите файл sales_data.xlsx
# Загрузка Excel
df = pd.read_excel("/content/sales_data.xlsx")
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Region,Item,Quantity
0,2024-01-01,North,Chair,25
1,2024-01-01,North,Table,72
2,2024-01-01,North,Desk,68
3,2024-01-01,North,Lamp,79
4,2024-01-01,North,Cabinet,88


In [14]:
# 📊 Парсинг данных
# TODO: используйте pandas для чтения и группировки по Region
regions = df['Region'].unique()
region_docs = {}

for region in regions:
    subset = df[df['Region'] == region]
    text = f"Sales data for region {region}:"
    for _, row in subset.iterrows():
        text += f"{row['Date'].date()} — {row['Item']}: {row['Quantity']} units\n"
    region_docs[region] = Document(text=text)

А нельзя было просто загрузить csv файл в векторную базу?

✅ Можно, но… вот нюансы:
📄 Если просто загрузить CSV как текст:

from llama_index.readers.file import CSVReader
reader = CSVReader()
documents = reader.load_data("sales_data.csv")

📌 Тогда весь CSV будет превращён в один большой документ, например в виде:

Date,Region,Item,Quantity
2024-01-01,North,Chair,50
2024-01-01,North,Table,23
...
Затем можно построить VectorStoreIndex на этом документе.

🤔 Тогда в чём проблема?
🔎 Гранулярность поиска:
Векторизация будет работать по чанкам из CSV — не по логике "1 регион = 1 блок", а по строчкам или кускам файла. Это затруднит задание точечных вопросов: "дай прогноз по региону West".

🧠 Контекст LLM будет мешаться:
Промпт получит мешанину строк по всем регионам. LLM может не уловить структуру, будет сложно интерпретировать "что где".

🕸️ Нет modular-логики:
Ты не сможешь запускать анализ по регионам в параллельных ветках (SubGraph), т.к. весь CSV — один документ.

🚀 Как правильно:
Если CSV = табличный фактологический источник (структурированное знание):
Разбей его на логические части (например, по регионам → Document), и только потом загружай в векторную БД.

Если CSV = простой справочник или словарь (типа описание продуктов):
Можно загрузить как один документ — проблем меньше.

💡 Вывод: да, CSV можно загрузить напрямую, но для задач типа прогнозов по регионам правильнее предобработать → логически разбить → индексировать по частям. Это даёт контролируемость и масштабируемость.









In [20]:
# 🧠 Индексация векторных данных
# TODO: создайте VectorStoreIndex с FAISS

# Определите размерность встраиваний, соответствующую вашей модели эмбеддингов
dimension = 1536  # Например, для модели OpenAI's text-embedding-ada-002
# Создайте индекс FAISS
faiss_index = faiss.IndexFlatL2(dimension)
# Инициализируйте FaissVectorStore с созданным индексом
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents=region_docs.values(), storage_context=storage_context)

engine = index.as_query_engine(llm=OpenAI(model="gpt-4o"))


In [21]:
# 🔁 Генерация прогнозов по регионам
# TODO: напишите цикл анализа и формирования markdown-отчета
for region, doc in region_docs.items():
    print(f"📍 Прогноз для {region}")
    response = engine.query(
        f"На основе данных, спрогнозируй спрос на товары в регионе {region} в июле 2024. Укажи тренды, какие товары растут/падают."
    )
    print(response.response)
    print("-" * 50)

📍 Прогноз для North
Для прогнозирования спроса на товары в регионе North в июле 2024 года можно рассмотреть данные за январь 2024 года. 

1. **Стулья (Chair):** Продажи варьировались, но в целом наблюдается тенденция к увеличению спроса, особенно в начале и середине месяца.
2. **Столы (Table):** Продажи колебались, но в целом спрос оставался стабильным с некоторыми пиками.
3. **Письменные столы (Desk):** Продажи показывают значительные колебания, но в целом спрос остается на высоком уровне.
4. **Лампы (Lamp):** Спрос на лампы также колебался, но в целом оставался стабильным.
5. **Шкафы (Cabinet):** Продажи шкафов показывают значительные колебания, но в целом спрос остается на высоком уровне.

Таким образом, можно ожидать, что спрос на стулья и письменные столы будет расти, в то время как спрос на столы, лампы и шкафы останется стабильным или будет незначительно колебаться.
--------------------------------------------------
📍 Прогноз для South
Для прогнозирования спроса на товары в реги

In [22]:
# 📤 Вывод результата
# TODO: сохраните итог в файл или отобразите в markdown формате
forecast_md = ""

for region in region_docs:
    response = engine.query(
        f"На основе данных, спрогнозируй спрос на товары в регионе {region} в июле 2024. Укажи тренды, что растёт или падает."
    )
    forecast_md += f"## 📍 Регион: {region}\n\n"
    forecast_md += response.response + "\n\n---\n\n"

# Сохраняем в markdown-файл
with open("forecast_report.md", "w", encoding="utf-8") as f:
    f.write(forecast_md)


✅ 1. faiss_index = faiss.IndexFlatL2(dimension)
📌 Что делает:
Создаёт пустой векторный индекс FAISS с метрикой L2 (евклидово расстояние) и заданной размерностью эмбеддингов.

🔎 Почему это нужно:
FAISS — это высокопроизводительная библиотека поиска по векторам. Мы используем её как backend-хранилище, куда будут записаны эмбеддинги документов.
dimension должен соответствовать размерности встраиваний модели эмбеддингов (например, 1536 для text-embedding-ada-002).

✅ 2. vector_store = FaissVectorStore(faiss_index=faiss_index)
📌 Что делает:
Оборачивает FAISS-объект в адаптер LlamaIndex, чтобы он мог использовать FAISS как VectorStore.

🔎 Почему это нужно:
LlamaIndex требует, чтобы все векторные БД были представлены через свой интерфейс VectorStore. Это позволяет LlamaIndex работать с разными хранилищами (FAISS, Chroma, Pinecone) единообразно.

✅ 3. storage_context = StorageContext.from_defaults(vector_store=vector_store)
📌 Что делает:
Создаёт объект StorageContext, который сообщает LlamaIndex, какое векторное хранилище использовать для индексации.

🔎 Почему это нужно:
StorageContext управляет слоями хранения: документами, векторами, метаданными. Передав vector_store, мы явно указываем, куда писать эмбеддинги.

✅ 4.

index = VectorStoreIndex.from_documents(
    documents=region_docs.values(),
    storage_context=storage_context
)
📌 Что делает:
Создаёт VectorStoreIndex, т.е. полноценный векторный индекс из набора документов. Автоматически:

Делает эмбеддинги для каждого документа.

Сохраняет их в FAISS.

Связывает документы с их эмбеддингами.

🔎 Почему это нужно:
Этот шаг позволяет нам впоследствии выполнять семантический поиск и RAG-запросы: мы можем спрашивать, и движок найдёт релевантные документы по смыслу (а не по ключевым словам).

🧩 В совокупности, эти строки:

Инициализируют FAISS как хранилище.

Оборачивают его для LlamaIndex.

Пропускают через него документы и формируют векторный индекс.

После этого можно делать:

response = index.as_query_engine().query("Что чаще всего покупали в регионе West?")
— и получить осмысленный ответ от LLM с опорой на твои данные.









---

### 🟨 Horizontal Scaling в LangGraph
- Разделение по SubGraphs
- Асинхронные `Node`
- Параллельный запуск функций анализа
- Поддержка ветвлений по регионам

---

In [23]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 13.8 MB/s eta 0:00:00


In [26]:
from langgraph.graph import StateGraph
from typing import TypedDict

In [29]:
class ForecastState(TypedDict):
    region: str
    forecast: str

def analyze_region(state):
    region = state["region"]
    response = engine.query(f"Спрогнозируй спрос на товары в регионе {region} в июле 2024.")
    return {"region": region, "forecast": response.response}

graph = StateGraph(state_schema=ForecastState)

for region in regions:
  node_name = f"region_{region}"
  graph.add_node(node_name, analyze_region)
  graph.set_entry_point(node_name)
  graph.set_finish_point(node_name)

compiled_graph = graph.compile()
compiled_graph.get_graph().draw_mermaid()


'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__([<p>__start__</p>]):::first\n\tregion_North(region_North)\n\tregion_South(region_South)\n\tregion_East(region_East)\n\tregion_West(region_West)\n\t__end__([<p>__end__</p>]):::last\n\t__start__ --> region_East;\n\t__start__ --> region_North;\n\t__start__ --> region_South;\n\t__start__ --> region_West;\n\tregion_East --> __end__;\n\tregion_North --> __end__;\n\tregion_South --> __end__;\n\tregion_West --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'

📊 Что ты сейчас видишь:
Это mermaid-диаграмма графа выполнения:

__start__ --> region_East
__start__ --> region_North
__start__ --> region_South
__start__ --> region_West

region_East --> __end__
region_North --> __end__
region_South --> __end__
region_West --> __end__

🧠 Что означает:
У тебя одновременный запуск всех 4 узлов.

Каждый region_X — это отдельная ветка анализа (Node).

Они асинхронны и независимы.

После выполнения — граф завершается (__end__).

✅ Что можно добавить:
Результаты собрать в список:
Использовать gather_node, чтобы собрать всё в один forecast_list.

SubGraph на каждый регион:
Если логика анализа сложная — оборачивай в SubGraph.

Logging и Retry:
Через retry_node или langsmith-интеграцию.

Интеграция с Telegram/ноутбуком/таблицей:
Отправить результат анализа.

In [31]:
# Результаты собраны в список: Использовать gather_node, чтобы собрать всё в один forecast_list

class ForecastState(TypedDict):
    region: str
    forecast: str

def analyze_region(state):
    region = state["region"]
    response = engine.query(f"Спрогнозируй спрос на товары в регионе {region} в июле 2024.")
    return {"region": region, "forecast": response.response}

graph = StateGraph(state_schema=ForecastState)

for region in regions:
  node_name = f"region_{region}"
  graph.add_node(node_name, analyze_region)
  graph.set_entry_point(node_name)
  graph.set_finish_point(node_name)


def gather_forecasts(state):
    forecasts = state.get("forecast_list", [])
    forecasts.append({"region": state["region"], "forecast": state["forecast"]})
    return {"forecast_list": forecasts}

def print_forecasts(state):
    for item in state["forecast_list"]:
        print(f"📍 {item['region']}:\n{item['forecast']}\n---")
    return state

graph.add_node("gather_forecasts", gather_forecasts)
graph.add_node("print_forecasts", print_forecasts)

for region in regions:
    graph.add_edge(f"region_{region}", "gather_forecasts")

graph.add_edge("gather_forecasts", "print_forecasts")
graph.set_finish_point("print_forecasts")

compiled_graph = graph.compile()
compiled_graph.get_graph().draw_mermaid()

'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__([<p>__start__</p>]):::first\n\tregion_North(region_North)\n\tregion_South(region_South)\n\tregion_East(region_East)\n\tregion_West(region_West)\n\tgather_forecasts(gather_forecasts)\n\tprint_forecasts(print_forecasts)\n\t__end__([<p>__end__</p>]):::last\n\t__start__ --> region_East;\n\t__start__ --> region_North;\n\t__start__ --> region_South;\n\t__start__ --> region_West;\n\tgather_forecasts --> print_forecasts;\n\tregion_East --> gather_forecasts;\n\tregion_North --> gather_forecasts;\n\tregion_South --> gather_forecasts;\n\tregion_West --> gather_forecasts;\n\tprint_forecasts --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'


## 🔁 Дополнительно: Loops & Iteration в LangGraph

LangGraph поддерживает итеративную обработку через циклы в графе состояний. Например, можно реализовать цикл переформулировки запроса, если результат не устраивает.

Пример узла:
```python
def node(state):
    if state["attempts"] > 3:
        return {"status": "fail"}
    if "недостаточно данных" in state["response"]:
        state["attempts"] += 1
        state["query"] = improve_prompt(state["query"])
        return state
    return {"status": "ok", "response": state["response"]}
```


In [ ]:

# 🔁 TODO: пример цикла на основе анализа
# Эмулируем переформулировку запроса при плохом результате

def simulate_loop(query, max_attempts=3):
    attempt = 0
    while attempt < max_attempts:
        print(f"Попытка {attempt+1}: Запрос = {query}")
        response = "недостаточно данных" if attempt < 2 else "успешный ответ"
        print(f"Ответ: {response}")
        if "недостаточно данных" not in response:
            break
        query += " подробнее"
        attempt += 1

simulate_loop("Сколько потребуется товара A в июле?")


In [32]:
def node(state):
    if state["attempts"] > 3:
        return {"status": "fail"}
    if "недостаточно данных" in state["response"]:
        state["attempts"] += 1
        state["query"] = improve_prompt(state["query"])
        return state
    return {"status": "ok", "response": state["response"]}

In [33]:
# 🔁 TODO: пример цикла на основе анализа
# Эмулируем переформулировку запроса при плохом результате

def simulate_loop(query, max_attempts=3):
    attempt = 0
    while attempt < max_attempts:
        print(f"Попытка {attempt+1}: Запрос = {query}")
        response = "недостаточно данных" if attempt < 2 else "успешный ответ"
        print(f"Ответ: {response}")
        if "недостаточно данных" not in response:
            break
        query += " подробнее"
        attempt += 1

simulate_loop("Сколько потребуется товара A в июле?")

Попытка 1: Запрос = Сколько потребуется товара A в июле?
Ответ: недостаточно данных
Попытка 2: Запрос = Сколько потребуется товара A в июле? подробнее
Ответ: недостаточно данных
Попытка 3: Запрос = Сколько потребуется товара A в июле? подробнее подробнее
Ответ: успешный ответ


🧱 Архитектура
LangGraph = Graph + State + Nodes + Edges

1. State (состояние):
Обычный Python-словарь или TypedDict:

state = {
  "query": "Что купить в июле?",
  "response": "...",
  "attempts": 2
}
Ты сам решаешь, какие ключи будут в состоянии.

2. Node (узел):
Это функция, которая получает state, делает что-то и возвращает новое состояние.

def search_node(state):
    query = state["query"]
    result = do_search(query)
    return {"response": result}
    
3. Edges (переходы):
Указывают, в какой узел идти дальше, в зависимости от state.

graph.add_conditional_edges("search", condition_fn={
  "success": "summarize",
  "fail": "retry"
})

4. Graph (граф):
Ты добавляешь ноды и связи, потом компилируешь:

graph = StateGraph()
graph.add_node("search", search_node)
graph.add_node("summarize", summarize_node)
graph.set_entry_point("search")
graph.set_finish_point("summarize")
compiled = graph.compile()
compiled.invoke(state)



Это упражнение показывает, как реализовать цикл: "если в ответе недостаточно данных — перезапросить до 3 раз".

In [34]:
from langgraph.graph import StateGraph
from typing import TypedDict

class LoopState(TypedDict):
    query: str
    response: str
    attempts: int

def node(state: LoopState) -> LoopState:
    print(f"Запрос: {state['query']} | Попытка: {state['attempts']+1}")

    if state["attempts"] >= 2:
        return {"query": state["query"], "response": "успешный ответ", "attempts": state["attempts"] + 1}

    state["response"] = "недостаточно данных"
    state["attempts"] += 1
    state["query"] += " подробнее"
    return state

def check_status(state: LoopState):
    if "недостаточно данных" in state["response"]:
        return "retry"
    return "done"

def final(state: LoopState):
    print(f"✅ Финальный ответ: {state['response']}")
    return state

graph = StateGraph(state_schema=LoopState)
graph.add_node("node", node)
graph.add_node("final", final)
graph.add_conditional_edges("node", check_status, {
    "retry": "node",
    "done": "final"
})
graph.set_entry_point("node")
graph.set_finish_point("final")

compiled = graph.compile()

state = {"query": "Сколько нужно товара?", "response": "", "attempts": 0}
compiled.invoke(state)


Запрос: Сколько нужно товара? | Попытка: 1
Запрос: Сколько нужно товара? подробнее | Попытка: 2
Запрос: Сколько нужно товара? подробнее подробнее | Попытка: 3
✅ Финальный ответ: успешный ответ


{'query': 'Сколько нужно товара? подробнее подробнее',
 'response': 'успешный ответ',
 'attempts': 3}